# Setup Empty Pipeline with Screen


In [1]:
# import resources
from zipline.pipeline import Pipeline
from zipline.pipeline.factors.basic import SimpleMovingAverage
from zipline.pipeline.data import USEquityPricing

# Create a screen for our Pipeline
mean_close_10 = SimpleMovingAverage(
    inputs=[USEquityPricing.close],
    window_length=10
)

universe = mean_close_10 > 10

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen=universe)

# Setup Zipline Engine

In [2]:
import helper
from zipline.utils.calendars import get_calendar
from zipline.data import bundles

# Name of bundle
EOD_BUNDLE_NAME = 'quantopian-quandl'

# Load the data bundle
bundle_data = bundles.load(EOD_BUNDLE_NAME)

# Setup the engine to look at the top 500 stocks who have had the highest rolling Average Dollar Volume
# over a 120-day window -- This is arbitrary and we can use this parameter to refine which stocks we
# want in our universe
universe = mean_close_10.top(500) 

# Select the trading calendar that will be used as a reference when slicing the data
trading_calendar = get_calendar('NYSE') 

# Load the bundle we configured in the previous step into the engine
bundle_data = bundles.load(EOD_BUNDLE_NAME)

# Create the engine -- the details of this function are in the utils.py file
engine = helper.build_pipeline_engine(bundle_data, trading_calendar)

# Run Pipeline Engine

In [3]:
import pandas as pd

# Set the start and end dates
start_date = pd.Timestamp('2017-01-05', tz = 'utc')
end_date = pd.Timestamp('2017-01-05', tz = 'utc')

# Run our pipeline for the given start and end dates
pipeline_output = engine.run_pipeline(pipeline, start_date, end_date)

# Get Universe Tickers

In [4]:
# Get the values in index level 1 and save them to a list
universe_tickers = pipeline_output.index.get_level_values(1).values.tolist()

# Create Data Portal

In [5]:
from zipline.data.data_portal import DataPortal

# Create a data portal
data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = bundle_data.equity_daily_bar_reader.first_trading_day,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)

# View Historical Data

Get the OHLC + V data for a given time period. This data will be split into individual dataframes.

In [6]:
def get_field_data(data_portal, trading_calendar, assets, start_date, end_date, field):
    
    # Set the given start and end dates to Timestamps. The frequency string C is used to
    # indicate that a CustomBusinessDay DateOffset is used
    end_dt = pd.Timestamp(end_date, tz='UTC', freq='C')
    start_dt = pd.Timestamp(start_date, tz='UTC', freq='C')

    # Get the locations of the start and end dates
    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    # return the historical data for the given window
    return data_portal.get_history_window(assets=assets, end_dt=end_dt, bar_count=end_loc - start_loc,
                                          frequency='1d',
                                          field=field,
                                          data_frequency='daily')

# The window of data to obtain
start_date = '2012-01-05'
end_date = '2017-01-05'

# Get the open data
open_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                          start_date, end_date, 'open')

# Get the high data
high_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                              start_date, end_date, 'high')

# Get the low data
low_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                         start_date, end_date, 'low')

# Get the closing data
close_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                              start_date, end_date, 'close') 

# Get the volume data
volume_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                            start_date, end_date, 'volume')

from pypf.instrument import DataframeInstrument
from pypf.chart import PFChart

def generate_pf_chart(ticker, historical_dfs):
    ''' 
        This function will create a p&f chart for the given ticker using historical data
        
        @param ticker: (str) ticker of asset to create P&F chart for
        @param historical_dfs: (pd.DataFrame) DataFrame holding historical ticker data
        
        return: PFChart object from which P&F chart/data can be extracted
    
    '''
    # Set up dataframe instrument
    try:
        df = historical_dfs[ticker][0]
    except:
        raise ValueError('Ticker passed does not exist in historical dataset')

    # Format date and volume values
    df['Date'] = df.index.astype(str)
    df['Date'] = df['Date'].str.slice(0,10)
    df['Volume'] = df['Volume'].astype(int)

    # Tes
    dfi = DataframeInstrument(ticker, dataframe=df)

    # Create pf chart
    chart = PFChart(dfi, duration=5)
    chart.create_chart()
    return chart

# Combine OHLC dataframes into singular dataframe
Here we combine the four individual dataframes representing OHLC + V data into one historical dataframe.

In [7]:
import numpy as np

#Create dataframe and append blank row
historical_dfs = pd.DataFrame(columns=universe_tickers)
historical_dfs = historical_dfs.append(pd.Series([np.nan]), ignore_index=True)

# Loop through each universe ticker and create a combined dataframe for that ticker
for ticker in universe_tickers:
    # Get individual series representing the OHLCV data
    open_series = open_data[ticker]
    high_series = high_data[ticker]
    low_series = low_data[ticker]
    close_series = close_data[ticker]
    volume_series = volume_data[ticker]
    
    # Combine these series into 1 dataframe
    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.concat([open_series,high_series, low_series, close_series, volume_series], axis=1)
    df.columns = columns    
    
    # Save this dataframe to historical_dfs
    historical_dfs[ticker] = pd.Series([df])
    
    
# Change the columns to be more human readable
columns = helper.beautify_tickers(universe_tickers)
historical_dfs.columns = columns

# Format P&F Chart Data For Analysis
Create a new DataFrame to hold the P&F chart data to be used for model training datasets. 
Columns '50 Day Moving Average of Volume' and 'Ratio of Volume to The 50 Day Moving Average' are generated (Does not apply to first 50 datapoints). Categorical columns are refactored to be nomial integers.

In [8]:
def format_data(pf_chart):
    # Import security data as a dataframe
    pf_chart_data = pd.DataFrame.from_dict(pf_chart.chart_meta_data)
    pf_chart_data = pf_chart_data.transpose().reset_index(drop=True)

    # Create a 50 day moving average of the volume and a ratio between the volume and the 50 day moving average
    pf_chart_data['50ma'] = pd.Series.rolling(pf_chart_data['volume'],50, min_periods=50).mean().round()
    pf_chart_data['volume_ratio'] = (pf_chart_data['volume'] / pf_chart_data['50ma'])
    # Reformat categorical columns to be numeric (cant decide if these should be numbered by index or polarity ie: buy is 1, sell is -1, none is 0)
    pf_chart_data['action'] = pf_chart_data['action'].map({'none': 0, 'x': 1, 'reverse x->o': 2, 'o': 3, 'reverse o->x': 4})
    pf_chart_data['direction'] = pf_chart_data['direction'].map({'o': 0, 'x':1})
    pf_chart_data['signal'] = pf_chart_data['signal'].map({'sell': 0, 'buy':1}) # We may want to refactor this so none = NaN
    
    return pf_chart_data

# Test Classification and Regression With Output

In [9]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz

def CART_Output(title,df,X,Y,df_names,test_size,random_state,max_depth,min_samples_leaf):
    #define training and test samples
    X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = test_size, random_state = random_state)
    X_train = X_train.astype(int)
    y_train = y_train.astype(int)
    X_test = X_test.astype(int)
    y_test = y_test.astype(int)

    #predict using Gini results
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = random_state,
                                   max_depth=max_depth, min_samples_leaf=min_samples_leaf)
    clf_gini.fit(X_train, y_train)
    y_pred = clf_gini.predict(X_test)

    #redefine arrays as integer (otherwise error message results from accuracy calculation)
    y_test = y_test.astype(int)
    y_pred = y_pred.astype(int)

    print ("Values for factors for Gini -",title,":")
    print('')

    def measure_performance(X,y,clf_gini, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True):
        y_pred=clf_gini.predict(X)   
        if show_accuracy:
            print ("Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred)),"\n")

        if show_classification_report:
            print ("Classification report")
            print (metrics.classification_report(y,y_pred),"\n")

        if show_confusion_matrix:
            print ("Confusion matrix")
            print (metrics.confusion_matrix(y,y_pred),"\n")

    output = measure_performance(X_train,y_train,clf_gini, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True)
    feature_names=df_names
    with open("cart-models/"+title+"_"+str(test_size)+"_"+str(random_state)+"_"+str(max_depth)+"_"+str(min_samples_leaf)+".txt", "w") as f:
        f = tree.export_graphviz(clf_gini, feature_names=df_names, out_file=f)

# Test Classification and Regression (Dynamic Parameters)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
import graphviz

def CART_Test(title,df,X,Y,df_names,test_size,random_state,max_depth,min_samples_leaf):

    #define training and test samples
    X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = test_size, random_state = random_state)
    X_train = X_train.astype(int)
    y_train = y_train.astype(int)
    X_test = X_test.astype(int)
    y_test = y_test.astype(int)

    #predict using Gini results
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = random_state,
                                   max_depth=max_depth, min_samples_leaf=min_samples_leaf)
    clf_gini.fit(X_train, y_train)
    y_pred = clf_gini.predict(X_test)

    #redefine arrays as integer (otherwise error message results from accuracy calculation)
    y_test = y_test.astype(int)
    y_pred = y_pred.astype(int)

    def measure_performance(X,y,clf_gini, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True):
        y_pred=clf_gini.predict(X)   
        if show_accuracy:
            accuracy = metrics.accuracy_score(y,y_pred)
        if show_classification_report:
            classification_report = metrics.classification_report(y,y_pred)
        if show_confusion_matrix:
            confusion_matrix = metrics.confusion_matrix(y,y_pred)
        return accuracy, classification_report, confusion_matrix
    accuracy, classification_report, confusion_matrix = measure_performance(X_train,y_train,clf_gini, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True)
    report_entry = {
        'ticker': '',
        'test_size': test_size,
        'random_state': random_state, 
        'max_depth': max_depth,
        'min_samples_leaf': min_samples_leaf,
        'accuracy': accuracy,
        'classification_report': classification_report, 
        'confusion_matrix': confusion_matrix
    }
    return report_entry

In [11]:
import itertools
from IPython.display import clear_output

# Set the tickers and parameter values to be tested 
ticker_set = ['AAPL', 'MMM', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'XOM', 'GS', 'HD' , 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'UTX', 'UNH', 'VZ', 'V', 'WMT', 'WBA']
test_size_set = [.20,.25,.30] # Percent of the data that is used as training data (Range = 0:1) 
random_state_set = [0,100] # Percent of the data that is randomized in the training/testing data (Range = 0:100)
max_depth_set = [3,4,5] # Max number of layers in the model (Range = 1:inf)
min_samples_leaf_set = [5,15,30] # Minimum number of samples in a group to split on (Range = 1:inf)
 
accuracy_report = pd.DataFrame(columns = ['ticker', 'test_size', 'random_state', 'max_depth', 'min_samples_leaf', 'accuracy', 'classification_report', 'confusion_matrix'])
current_ticker = ''

# Test all combinations of the given objects
for ticker, test_size, random_state, max_depth, min_samples_leaf in itertools.product(ticker_set, test_size_set, random_state_set, max_depth_set, min_samples_leaf_set):
    if current_ticker != ticker:
        current_ticker = ticker
        clear_output()
        print ('On Ticker: ' + ticker)
    else:
        pass
    
    # Set input variables to test
    
    # Numeric values or column names of independent varibles
    input_cols = [1,5,6,8,9,10,15,16,17]
    # Numeric value or column name of the dependent variable
    output_col = 13
    # Title of the chart
    title = 'PnF of ' + ticker 

    # Create a p&f chart for the given ticker using historical data and print the result
    pf_chart = generate_pf_chart(ticker, historical_dfs)
    # Format P&F Chart Data For Analysis
    pf_chart_data = format_data(pf_chart)
    # Set the DataFrame to test
    df = pf_chart_data
    
    # Clean dataset of rows that contain 'nan'
    df.dropna(inplace = True, axis = 'rows')
    
    #Set the range of the test ()
    X = df.values[:len(df)-1,(input_cols)]
    Y = df.values[1:len(df),output_col]


    # Run the CART model generator
    df_names = list(df.columns.values[input_cols])
    report_entry = CART_Test(title,df,X,Y,df_names,test_size,random_state,max_depth,min_samples_leaf)
    report_entry['ticker'] = ticker
    accuracy_report = accuracy_report.append(report_entry, ignore_index=True)

# Reorder accuracy report descending by accuracy
accuracy_report = accuracy_report.sort_values(by='accuracy',ascending=False).reset_index(drop=True)
clear_output()
print('Done!')

Done!


In [17]:
ticker_report = accuracy_report.drop_duplicates(subset = 'ticker')
for index, row in ticker_report.iterrows():
    ticker = row['ticker']
    test_size = row['test_size']
    max_depth = row['max_depth']
    random_state = row['random_state']
    min_samples_leaf = row['min_samples_leaf']
    min_samples_leaf = row['min_samples_leaf']
    
    # Recreate the analysis DataFrame using the best performer
    pf_chart = generate_pf_chart(ticker, historical_dfs)
    pf_chart_data = format_data(pf_chart)
    df = pf_chart_data
    df.dropna(inplace = True, axis = 'rows')
    X = df.values[:len(df)-1,(input_cols)]
    Y = df.values[1:len(df),output_col]

    # Run the CART model generator
    title = 'PnF of ' + ticker 
    df_names = list(df.columns.values[input_cols])

    CART_Output(title,df,X,Y,df_names,test_size,random_state,max_depth,min_samples_leaf) 
clear_output()
ticker_report

,ticker,test_size,random_state,max_depth,min_samples_leaf,accuracy,classification_report,confusion_matrix
0,JNJ,0.20,100,5,5,0.950216,precision recall f1-score s...,"[[189, 20], [26, 689]]"
3,PG,0.30,0,5,5,0.946682,precision recall f1-score s...,"[[251, 14], [31, 548]]"
4,WMT,0.20,0,5,5,0.946203,precision recall f1-score s...,"[[381, 32], [19, 516]]"
14,XOM,0.30,0,5,5,0.931280,precision recall f1-score s...,"[[302, 45], [13, 484]]"
16,MMM,0.30,0,5,5,0.930095,precision recall f1-score s...,"[[175, 42], [17, 610]]"
19,DIS,0.20,100,5,5,0.929534,precision recall f1-score s...,"[[188, 63], [5, 709]]"
25,GS,0.30,0,5,5,0.927725,precision recall f1-score s...,"[[240, 24], [37, 543]]"
27,CVX,0.30,100,5,5,0.926540,precision recall f1-score s...,"[[312, 12], [50, 470]]"
36,MCD,0.30,0,5,5,0.921687,precision recall f1-score s...,"[[328, 41], [24, 437]]"
74,KO,0.30,0,5,5,0.905213,precision recall f1-score s...,"[[340, 34], [46, 424]]"


# Test 6 ML models to compare accuracies
Using the pf_chart_data DataFrame's independent variables, different non-linear functions can be quickly tested to gauge general accuracy in predicting the dependent variable, buy_sell_col. The box and whisker plots provided in the output graph the different results as well as their accuracy spreads.

In [ ]:
# Compare Algorithms
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()
# TODO: Add confusion matrices